In [6]:
!jupyter nbextension enable --py --sys-prefix pythreejs
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-threejs/extension...
      - Validating: OK
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [7]:
import numpy as np
from pythreejs import (Renderer, Scene, PerspectiveCamera, Mesh, BoxGeometry, SphereGeometry, CylinderGeometry, ConeGeometry, PlaneGeometry,
                       MeshStandardMaterial, DirectionalLight, OrbitControls, BufferGeometry, CircleGeometry, RingGeometry, GridHelper)
from scipy.spatial.transform import Rotation as R
from IPython.display import display

In [8]:
class GeometricPrimitive:
    def __init__(self, geometry, position, name, material_type, group_number, emissivity, specularity, rotation=None):
        print("Creating GeometricPrimitive object:"+name)
        # return error if geometry is not a pythreejs geometry object
        if not isinstance(geometry, (SphereGeometry, CylinderGeometry, CircleGeometry, RingGeometry, ConeGeometry, PlaneGeometry)):
            raise ValueError("geometry must be a supported pythreejs geometry object")
        # return error if position is not of length 3
        if not len(position) == 3:
            raise ValueError("position must be a list of length 3")
        # rotation must be None or length 3
        # if rotation is not None or not len(rotation) == 3:
        #     raise ValueError("rotation must be a list of length 3")
        # return error if name is not a string of less than 30 characters
        if not isinstance(name, str) or len(name) > 30:
            raise ValueError("name must be a string of less than 30 characters")
        # return error if material type is not 1 or 0
        if material_type not in [0, 1]:
            raise ValueError("material_type must be 0 or 1")
        # return error if group number is not an integer
        if not isinstance(group_number, int):
            raise ValueError("group_number must be an integer")
        # return error if emissivity is not a float between 0 and 1
        if not isinstance(emissivity, float) or emissivity < 0 or emissivity > 1:
            raise ValueError("emissivity must be a float between 0 and 1")
        # return error if specularity is not a float between 0 and 1
        if not isinstance(specularity, float) or specularity < 0 or specularity > 1:
            raise ValueError("specularity must be a float between 0 and 1")

        self.geometry = geometry
        self.position = position
        self.rotation= rotation
        self.name = name
        self.material_type = material_type
        self.group_number = group_number #component group number
        self.emissivity = emissivity 
        self.specularity = specularity
        self.calculate_properties()

        # if the object is a cylinder, add two circles as the cylinder must be capped with circles at each end...
        if isinstance(self.geometry, CylinderGeometry):
            # only allow cylinders that have a radius at the top and bottom that are equal
            if self.geometry.radiusTop != self.geometry.radiusBottom:
                raise ValueError("Cylinder radiusTop and radiusBottom must be equal")
            self.caps = [GeometricPrimitive(CircleGeometry(radius=self.radius), self.center_bot, self.name + '_base', self.material_type, self.group_number, self.emissivity, self.specularity),
                         GeometricPrimitive(CircleGeometry(radius=self.radius), self.center_top, self.name + '_top', self.material_type, self.group_number, self.emissivity, self.specularity)]
        
    def calculate_properties(self):
        if isinstance(self.geometry, BoxGeometry):
            self.vertices = []
            for i in range(-1, 2, 2):
                for j in range(-1, 2, 2):
                    for k in range(-1, 2, 2):
                        self.vertices.append([i * self.geometry.width / 2, j * self.geometry.height / 2, k * self.geometry.depth / 2])

        elif isinstance(self.geometry, SphereGeometry):
            self.radius = self.geometry.radius

        elif isinstance(self.geometry, CylinderGeometry):
            self.radius = self.geometry.radiusBottom
            # calculate rotation matrix of the cylinder
            if self.rotation is not None:
                r = R.from_euler('XYZ', self.rotation, degrees=False)
                self.rotation_matrix = r.as_matrix()
            else:
                self.rotation_matrix = np.identity(3)
            # calculate the unit verctors for the top and bottom of the cube
            z_axis = np.array([0, 0, 1])
            if self.rotation is not None:
                z_axis = np.matmul(self.rotation_matrix, z_axis)
            else:
                z_axis = np.array([0, 0, 1])
            self.base_unit_vector = -z_axis
            self.center_top = self.position + self.base_unit_vector * self.geometry.height / 2
            self.center_bot = self.position - self.base_unit_vector * self.geometry.height / 2
        
        elif isinstance(self.geometry, CircleGeometry):
            self.radius = self.geometry.radius
            
            def circum_point(theta): #calculate the coordinates of a point on the circumference of the circle
                return [self.radius * np.cos(theta), self.radius * np.sin(theta), 0]
            
            if self.rotation is not None: # apply a rotation matrix to the points if the circle is rotated
                r = R.from_euler('XYZ', self.rotation, degrees=False)
                self.rotation_matrix = r.as_matrix()
                rot_circum_point = lambda theta: np.matmul(self.rotation_matrix, circum_point(theta))
                self.circum_point1 = rot_circum_point(np.radians(45))
                self.circum_point2 = rot_circum_point(np.radians(135))
            else:
                self.circum_point1 = circum_point(np.radians(45))
                self.circum_point2 = circum_point(np.radians(135))
            
        
        elif isinstance(self.geometry, RingGeometry):
            self.outer_radius = self.geometry.outerRadius
            self.inner_radius = self.geometry.innerRadius
            self.thickness = self.outer_radius - self.inner_radius
        
        elif isinstance(self.geometry, ConeGeometry):
            self.depth = self.geometry.height
            self.radius_bottom = self.geometry.radius
            self.radius_top = 0 if not hasattr(self.geometry, 'radiusTop') else self.geometry.radiusTop
            self.base_center_coords = [0, -self.geometry.height / 2, 0]
            self.tip_coords = [0, self.geometry.height / 2, 0]
        
        elif isinstance(self.geometry, PlaneGeometry):
            self.vertices = []
            for i in range(-1, 2, 2):
                for j in range(-1, 2, 2):
                    self.vertices.append([i * self.geometry.width / 2, j * self.geometry.height / 2, 0])

def create_mesh(primitive, material):
    geometry = primitive.geometry
    mesh = Mesh(geometry=geometry, material=material)
    mesh.position = primitive.position
    if primitive.rotation is not None:
        r = R.from_euler('XYZ', primitive.rotation, degrees=False)
        mesh.quaternion = tuple(r.as_quat())
    return mesh

def setup_scene(primitives,rotations=None):
    camera = PerspectiveCamera(position=[3, 3, 3], aspect=1.0)
    controls = OrbitControls(controlling=camera)
    light = DirectionalLight(position=[10, 10, 10])

    scene = Scene()
    scene.add(camera)
    scene.add(light)
    
    material = MeshStandardMaterial(color='green')

    for i, primitive in enumerate(primitives):
        mesh = create_mesh(primitive, material)
        scene.add(mesh)

    renderer = Renderer(scene, camera, [controls], width=800, height=600)
    return renderer

def orientation2euler(bond_orientation, axis=2):
    bond_matrix = np.eye(3)
    bond_matrix[:, axis] = bond_orientation
    permutation = (axis + np.arange(3)) % 3
    bond_matrix = bond_matrix[:, permutation]
    q, r = np.linalg.qr(bond_matrix)
    if np.sign(r[0, 0]) == -1:
        bond_rotation_matrix = -q[:, [0, 2, 1]]
    else:
        bond_rotation_matrix = q
    permutation = np.arange(3)
    permutation[(axis + np.arange(3)) % 3] = np.arange(3)
    bond_rotation_matrix = bond_rotation_matrix[:, permutation]
    euler_rotation = R.from_matrix(bond_rotation_matrix).as_euler('XYZ', degrees=False)
    return euler_rotation

def plot_primitives(primitives):
    renderer = setup_scene(primitives)
    #add a grid in the background
    grid = GridHelper(size=10, divisions=10)
    renderer.scene.add(grid)
    display(renderer)

In [9]:
def primitives_to_ucl(primitives, ucl_filename):
    #convert primitives to ucl format according to the following rules

    # Each component record in the spacecraft description file begins with a header line specifying the type of component being described by that record (e.g. GENERAL, SPHERE, CONEI etc.). Thereafter, there is a comment line consisting of a record number, two solidi (//), a material type number, a component group number, and possibly a comment about the component, e.g.,
    # 3 // 001 0010 main bus +Z face
    # In this example the record number is 3; the material type number is 1 (any leading zeros are ignored); the component group number is 10; and, the comment provides some information about the component.
    # material type 1 is "MLI"
    # material type 0 is "not MLI"

    # PlanarPolygons ("GENERAL" in ucl, "PlaneGeometry" in pythreejs)
    # Consecutive lines of the component record specify geometrical attributes and are dependent on the geometrical primitve type.
    # for a PlaneGeometry ("GENERAL" in ucl), the first line specifies the number of vertices, 
    # the next lines specify the x, y, and z coordinates of a vertex of the plane, which are listed in anticlockwise order when viewed from the positive side of the plane 
    # the final line specifies the reflectivity and specularity coefficents respectively (0-1)

    # PlanarCircles ("CIRCLE" in ucl, "CircleGeometry" in pythreejs)
    # following the comment line, the radius of the circle is specified on the next line
    #the next lines are the positions of the center of the circle and two points on the circumference of the circle (also in anticlockwise order when viewed from the positive side of the circle)
    
    # Cylinders ("CYL_X" in ucl, "CylinderGeometry" in pythreejs)
    # following the comment line, the radius of the cross section of the cylinder is specified on the next line
    # the next lines are the postiions of the end points of the cylinder (center of the cylinder is the midpoint of the end points)
    # for cylinders to appear closed in the ucl viewer, the cylinder must be capped with circles at each end

    print(f"writing ucl file to {ucl_filename}...")
    print(f"number of primitives: {len(primitives)}")
    
    all_ucl = ""
    for i, primitive in enumerate(primitives):
        record_number = i + 1

        def comment_line(primitive,record_number):
            comment = f"{record_number} // {primitive.material_type} {primitive.group_number} {primitive.name}"
            return comment

        def emiss_spec_line(primitive):
            return f"{primitive.emissivity} {primitive.specularity}"

        if isinstance(primitive.geometry, PlaneGeometry):
            print(f"primitive {i} is a PlaneGeometry")
            ucl_format = "GENERAL\n"
            ucl_format += comment_line(primitive,record_number)+"\n"
            ucl_format += f"{len(primitive.vertices)}\n" #number of vertices
            for vertex in primitive.vertices: # specify the x, y, and z coordinates of a vertex of the plane (listed in anticlockwise order when viewed from the positive side of the plane)
                ucl_format += f"{vertex[0]} {vertex[1]} {vertex[2]}" + "\n"
            ucl_format += emiss_spec_line(primitive) + "\n"
            all_ucl += ucl_format

        elif isinstance(primitive.geometry, CircleGeometry):
            print(f"primitive {i} is a CircleGeometry")
            ucl_format = "CIRCLE\n"
            ucl_format += comment_line(primitive,record_number)+"\n"
            ucl_format += f"{primitive.radius}\n" #radius
            ucl_format += f"{primitive.position[0]} {primitive.position[1]} {primitive.position[2]}\n" #center of circle
            ucl_format += f"{primitive.circum_point1[0]} {primitive.circum_point1[1]} {primitive.circum_point1[2]}\n" # 1st point on circumference
            ucl_format += f"{primitive.circum_point2[0]} {primitive.circum_point2[1]} {primitive.circum_point2[2]}\n" # 2nd point on circumference
            ucl_format += emiss_spec_line(primitive) + "\n"
            all_ucl += ucl_format
            
        elif isinstance(primitive.geometry, CylinderGeometry):
            print(f"primitive {i} is a CylinderGeometry")
            ucl_format = "CYLINDER\n"
            ucl_format += comment_line(primitive,record_number)+"\n"
            # radius of cross section
            ucl_format += f"{primitive.radius}\n"
            # position of end points of cylinder
            ucl_format += f"{primitive.center_bot[0]} {primitive.center_bot[1]} {primitive.center_bot[2]}\n"
            ucl_format += f"{primitive.center_top[0]} {primitive.center_top[1]} {primitive.center_top[2]}\n"
            all_ucl += ucl_format

    #write each new instance of a primitive to a new line in the ucl file
    with open(ucl_filename, "a") as f:
        # clear the file
        f.truncate(0)
        # write the ucl format to the file
        f.write(all_ucl)


In [10]:
# bus = GeometricPrimitive(BoxGeometry(width=1, height=1, depth=3), position=[0, 0, 0], name = "potato bus", material_type = 1, group_number = 1, emissivity = 0.2, specularity = 0.1)
# array1 = GeometricPrimitive(BoxGeometry(width=5, height=0.05, depth=1), position=[0, 0, 0], name = "potato array1", material_type = 0, group_number = 3, emissivity = 0.3, specularity = 0.4)
bus_face_y_pos = GeometricPrimitive(PlaneGeometry(width=1, height=3), position=[0, 0.5, 0], name = "potato bus_face_y+", material_type = 0, group_number = 1, emissivity = 0.2, specularity = 0.1)
bus_face_y_neg = GeometricPrimitive(PlaneGeometry(width=1, height=3), position=[0, -0.5, 0], name = "potato bus_face_y-", material_type = 0, group_number = 1, emissivity = 0.2, specularity = 0.1)
bus_face_x_pos = GeometricPrimitive(PlaneGeometry(width=3, height=1), position=[0.5, 0, 0], name = "potato bus_face_x+", material_type = 0, group_number = 1, emissivity = 0.2, specularity = 0.1)
bus_face_x_neg = GeometricPrimitive(PlaneGeometry(width=3, height=1), position=[-0.5, 0, 0], name = "potato bus_face_x-", material_type = 0, group_number = 1, emissivity = 0.2, specularity = 0.1)
bus_face_z_pos = GeometricPrimitive(PlaneGeometry(width=1, height=3), position=[0, 0, 1.5], name = "potato bus_face_z+", material_type = 0, group_number = 1, emissivity = 0.2, specularity = 0.1)
bus_face_z_neg = GeometricPrimitive(PlaneGeometry(width=1, height=3), position=[0, 0, -1.5], name = "potato bus_face_z-", material_type = 0, group_number = 1, emissivity = 0.2, specularity = 0.1)

array1_face_y_pos = GeometricPrimitive(PlaneGeometry(width=5, height=1), position=[0, 0.05, 0], name = "potato array1_face_y+", material_type = 0, group_number = 3, emissivity = 0.3, specularity = 0.4)
array1_face_y_neg = GeometricPrimitive(PlaneGeometry(width=5, height=1), position=[0, -0.05, 0], name = "potato array1_face_y-", material_type = 0, group_number = 3, emissivity = 0.3, specularity = 0.4)
array1_face_x_pos = GeometricPrimitive(PlaneGeometry(width=1, height=5), position=[2.5, 0, 0], name = "potato array1_face_x+", material_type = 0, group_number = 3, emissivity = 0.3, specularity = 0.4)
array1_face_x_neg = GeometricPrimitive(PlaneGeometry(width=1, height=5), position=[-2.5, 0, 0], name = "potato array1_face_x-", material_type = 0, group_number = 3, emissivity = 0.3, specularity = 0.4)
array1_face_z_pos = GeometricPrimitive(PlaneGeometry(width=5, height=1), position=[0, 0, 0.5], name = "potato array1_face_z+", material_type = 0, group_number = 3, emissivity = 0.3, specularity = 0.4)
array1_face_z_neg = GeometricPrimitive(PlaneGeometry(width=5, height=1), position=[0, 0, -0.5], name = "potato array1_face_z-", material_type = 0, group_number = 3, emissivity = 0.3, specularity = 0.4)

antenna1 = GeometricPrimitive(CylinderGeometry(radiusTop=0.02, radiusBottom=0.02, height=0.4), position=[-0.3, 0.5, 1], name = "potato antenna1", material_type = 0, group_number = 2, emissivity = 0.4, specularity = 0.8)
antenna2 = GeometricPrimitive(CylinderGeometry(radiusTop=0.02, radiusBottom=0.02, height=0.4), position=[0.3, 0.5, 1], name = "potato antenna2", material_type = 0, group_number = 2, emissivity = 0.4, specularity = 0.8)
antenna3 = GeometricPrimitive(CylinderGeometry(radiusTop=0.02, radiusBottom=0.02, height=0.4), position=[0.0, 0.5, 1], name = "potato antenna3", material_type = 0, group_number = 2, emissivity = 0.4, specularity = 0.8)
truncated_paraboloid = GeometricPrimitive(CylinderGeometry(radiusTop=0.2, radiusBottom=0.2, height=0.2), position=[0, 0.5, -1], name = "potato paraboloid", material_type = 0, group_number = 4, emissivity = 0.2, specularity = 0.2)
rotated_cylinder = GeometricPrimitive(CylinderGeometry(radiusTop=0.05, radiusBottom=0.05, height=1.5), position=[0, 0.5, 1.4], rotation=orientation2euler(np.array([280, 10, 30])), name = "potato rotated cylinder", material_type = 0, group_number = 5, emissivity = 0.2, specularity = 0.2)

Creating GeometricPrimitive object:potato antenna1
Creating GeometricPrimitive object:potato antenna1_base
Creating GeometricPrimitive object:potato antenna1_top
Creating GeometricPrimitive object:potato antenna2
Creating GeometricPrimitive object:potato antenna2_base
Creating GeometricPrimitive object:potato antenna2_top
Creating GeometricPrimitive object:potato antenna3
Creating GeometricPrimitive object:potato antenna3_base
Creating GeometricPrimitive object:potato antenna3_top
Creating GeometricPrimitive object:potato paraboloid
Creating GeometricPrimitive object:potato paraboloid_base
Creating GeometricPrimitive object:potato paraboloid_top
Creating GeometricPrimitive object:potato rotated cylinder
Creating GeometricPrimitive object:potato rotated cylinder_base
Creating GeometricPrimitive object:potato rotated cylinder_top


In [11]:
sc_primitives = [antenna1, antenna2, antenna3, truncated_paraboloid, rotated_cylinder]
# plot_primitives(sc_primitives)
primitives_to_ucl(sc_primitives, "/Users/charlesc/Documents/GitHub/SRP_tools/potato_sc.txt")

writing ucl file to /Users/charlesc/Documents/GitHub/SRP_tools/potato_sc.txt...
number of primitives: 5
primitive 0 is a CylinderGeometry
primitive 1 is a CylinderGeometry
primitive 2 is a CylinderGeometry
primitive 3 is a CylinderGeometry
primitive 4 is a CylinderGeometry


In [4]:
#test above function by creating two planes and saving it to a ucl file
plane = GeometricPrimitive(geometry=PlaneGeometry(width=1, height=1), position=[0, 0, 0], rotation=None, name="test plane1", material_type=1, group_number=1, emissivity=0.5, specularity=0.5)
plane2 = GeometricPrimitive(geometry=PlaneGeometry(width=1, height=1), position=[0, 0, 1], rotation=None, name="test plane2", material_type=1, group_number=1, emissivity=0.2, specularity=0.1)
circle = GeometricPrimitive(geometry=CircleGeometry(radius=2), position=[1, 2, 1], rotation=None, name="test circle", material_type=1, group_number=2, emissivity=0.3, specularity=0.8)
test_prim_planes = [plane, plane2, circle]

In [5]:
primitives_to_ucl(test_prim_planes, "/Users/charlesc/Documents/GitHub/SRP_tools/test_sc_primitives.txt")

writing ucl file to /Users/charlesc/Documents/GitHub/SRP_tools/test_sc_primitives.txt...
number of primitives: 3
primitive 0 is a PlaneGeometry
primitive 1 is a PlaneGeometry
primitive 2 is a CircleGeometry


In [ ]:
#TODO: 
# function to convert from list of primitives to the UCL format (ensuring the rotations are applied correctly to the vertices as well)
# first -> list of required info for each shape from the manual (including material properties-change object colour as function of this)
# second -> convert each primitives definition to outlined requirements
# third -> apply rotation where/if necessary (apply rotation to the calculate_properties functon)

In [9]:
bus = GeometricPrimitive(BoxGeometry(width=1, height=1, depth=3), position=[0, 0, 0], name = "bus")
array1 = GeometricPrimitive(BoxGeometry(width=5, height=0.05, depth=1), position=[0, 0, 0], name = "array1")
antenna1 = GeometricPrimitive(CylinderGeometry(radiusTop=0.02, radiusBottom=0.05, height=0.4), position=[-0.3, 0.5, 1], name = "antenna1")
antenna2 = GeometricPrimitive(CylinderGeometry(radiusTop=0.02, radiusBottom=0.05, height=0.4), position=[0.3, 0.5, 1], name = "antenna2")
antenna3 = GeometricPrimitive(CylinderGeometry(radiusTop=0.02, radiusBottom=0.05, height=0.4), position=[0.0, 0.5, 1], name = "antenna3")
truncated_paraboloid = GeometricPrimitive(CylinderGeometry(radiusTop=0.4, radiusBottom=0.2, height=0.2), position=[0, 0.5, -1], name = "truncated_paraboloid")
rotated_cylinder = GeometricPrimitive(CylinderGeometry(radiusTop=0.05, radiusBottom=0.05, height=1.5), position=[0, 0.5, 1.4], rotation=orientation2euler(np.array([280, 10, 30])), name = "rotated_cylinder")

sc_primitives = [bus, array1, antenna1, antenna2, antenna3, truncated_paraboloid, rotated_cylinder]

plot_primitives(sc_primitives)

In [ ]:

    # TODO: make a function that takes in a list of primitives and returns a list of coords/info
    # for primitive in primitives:
    #     # ...
    #     if isinstance(primitive.geometry, BoxGeometry):
    #         print("Box vertices:")
    #         for vertex in primitive.vertices:
    #             print(vertex)
    #     elif isinstance(primitive.geometry, SphereGeometry):
    #         print(f"Sphere radius: {primitive.radius}, center: {primitive.position}")
    #     elif isinstance(primitive.geometry, CylinderGeometry):
    #         print(f"Cylinder radius: {primitive.radius}, base coordinates: {primitive.base_coords}, top coordinates: {primitive.top_coords}")
    #     elif isinstance(primitive.geometry, CircleGeometry):
    #         print(f"Circle radius: {primitive.radius}, center: {primitive.position}")
    #     elif isinstance(primitive.geometry, RingGeometry):
    #         print(f"Ring outer radius: {primitive.outer_radius}, inner radius: {primitive.inner_radius}, thickness: {primitive.thickness}, center: {primitive.position}")
    #     elif isinstance(primitive.geometry, ConeGeometry):
    #         if primitive.radius_top == 0:
    #             print(f"Paraboloid depth: {primitive.depth}, radius: {primitive.radius_bottom}, apex coordinates: {primitive.tip_coords}, base center coordinates: {primitive.base_center_coords}")
    #         else:
    #             print(f"Truncated Cone depth: {primitive.depth}, radius at the base: {primitive.radius_bottom}, tip radius: {primitive.radius_top}, base center coordinates: {primitive.base_center_coords}, tip coordinates: {primitive.tip_coords}")


In [ ]:
# create primitives, position them, apply rotation to mesh, apply rotation to vertices determining shape